# Calculs sur un Modèle Numérique de Terrain (MNT) 
# Projet deuxième partie - Implémentation

## M1 informatique, Université d'Orléans 2021/2022

**Ce projet est à faire en binôme. Vous aurez plusieurs rendus à déposer sur Celene mais avec un seul rendu par binôme.**

Il s'agit désormais d'implémenter le calcul des directions et le calcul des flots d'accumulation sur un MNT. Cette implémentation va être découpée en 3 parties avec une première étape pour distribuer correctement le MNT initial.

### 0. MPI et OpenMP
Il est possible d'écrire un programme SPMD en MPI qui permet d'intégrer des directives OpenMP afin de paralléliser sur différents coeurs du processus une partie du programme. 

Pour cela il faut modifier l'initialisation du contexte MPI en utilisant la fonction ci-dessous :


In [ ]:
int pid, nprocs;
int provided;
MPI_Init_thread(&argc, &argv, MPI_THREAD_MULTIPLE, &provided);
MPI_Comm_rank(MPI_COMM_WORLD, &pid);
MPI_Comm_size(MPI_COMM_WORLD, &nprocs);

### 1. La lecture du MNT
Sur Celene vous disposez d'un premier programme principal qui implémente la lecture d'un MNT dans un fichier .txt tel que décrit auparavant. Cette lecture est basée sur une classe **mnt** qui permet de construire un terrain avec tous les paramètres dont vous aurez besoin par la suite.
Dans cette archive vous trouverez également un dossier **Data** qui contient deux jeux de données **alpes.txt** un mnt de taille $1024 \times 1024$ et **jeu_essai.txt** de taille $32 \times 16$ qui vous permettra de tester dans un premier temps vos implémentations.

Le programme proposé prend en paramètres le nom du fichier contenant le MNT et l'identifiant du processus root.

### 2. La distribution des données 
Avant de mettre en place votre parallélisation, il est nécessaire que le processus **root** distribue le terrain qu'il a lu sur les différents processus de l'exécution paralléle. Pour cela vous allez pour répartir les données utiliser une distribution round-robin c'est-à-dire une distribution par bandes du MNT de manière circulaire sur les processus. Vous devez suivre les règles suivantes
1. la taille des bandes $t$ c'est-à-dire le nombre de lignes d'une bande du MNT sera donnée en argument et sera calculée de manière à ce que le nombre de lignes du MNT soit bien divisible par la taille des bandes
2. le nombre de bandes à distribuer est divisible par le nombre de processus (il n'y a pas de reste à distribuer)
3. comme le nombre de lignes du MNT est une puissance de 2, vous pourrez choisir une taille de bandes également en puissance de 2.

Dans le cas général, lorsqu'on calcule la direction d'un point du MNT il faut accéder à ses 8 voisins. Il est donc nécessaire de définir des ghosts qui correspondent à la ligne qui précède et qui suit la bande qu'un processus gère. La distribution doit tenir compte de la réception de ces valeurs supplémentaires. Vous devrez donc créer un tableau de taille $T$ tel que 
$$T = (t+2)\times m \times b$$
avec $t$ le nombre de lignes dans une bande, $m$ le nombre de colonnes du MNT soit la taille d'une ligne et $b$ le nombre de bandes par processus.

Implémentez une fonction qui réalise la distribution en round-robin du MNT tenant compte des ghosts sachant que c'est le processus **root** qui a lu le fichier de données pour construire un objet de la classe **mnt**.


In [ ]:
void round_robin(mnt* mnt, float* terrain_local, int taille_bande, int nb_lignes, int nb_cols, int root);

Attention l'allocation mémoire pour *terrain_local* qui va permettre de recevoir les bandes plus les ghosts est faite avant l'appel de la fonction. On suppose égalemet que les fonctions *MPI_Comm_rank* et *MPI_Comm_size* seront appelées dans la fonction pour connaître l'identifiant de chaque processus.
Attention également car la première bande et la dernière sont des cas particuliers par rapport aux ghosts.


**A rendre à 12h30 (dépôt RoundRobinRendu2) : Une archive .tgz contenant l'ensemble de vos codes avec le Makefile et également un README indiquant le travail réalisé.**

### 3. Le calcul des directions

Une fois les données distribuées, chaque processus peut calculer pour chaque point des bandes qu'il a reçues le codage de la direction vers le point le plus bas. Grâce aux ghosts, les voisins de tous les points sont accessibles. Il faudra quand même faire attention au traitement des points sur les bords verticaux (à droite et à gauche de la bande).

Implémentez la fonction suivante sachant que pour un point lorsque la valeur de hauteur est **NO_DATA** on ne calcule pas de direction ou on ne tient pas compte de ce voisin. Il est possible d'optimiser cette partie en utilisant **des directives OpenMP** afin de paralléliser par exemple la boucle de traitement des points.

Si vous souhaitez, vous disposez sur Celene d'une correction de la distribution round-robin. Vous êtes libre de l'utiliser ou pas.

In [ ]:
void calcul_direction(float *terrain_local, int *dir, int nb_bandes, int taille_bande, int nb_cols, float no_value);

Attention l'allocation mémoire du tableau *dir* sera faite à l'extérieur de la fonction. Il est également nécessaire de réfléchir à la taille de ce tableau qui sera nécessaire pour le calcul du flot d'accumulation.

**A rendre à 15h30 (dépôt DirectionRendu3) : Une archive .tgz contenant à nouveau l'ensemble de vos codes avec un README indiquant le travail réalisé et comment vous avez optimisé ce calcul sur chaque processus.**

### 4. Le calcul du flot d'accumulation

A partir de l'algorithme décrit dans la première partie, vous devez implémenter le calcul du flot d'accumulations utilisant le calcul des directions effectué par chacun des processus. Vous êtes libre d'implémenter cette partie comme vous le souhaitez mais
1. Initialement tous les points gérés par un processus doivent connaître la direction de tous ses voisins. Il est sans doute nécessaire de faire des échanges entre processus pour avoir toutes les informations.
2. L'algorithme est itératif et là encore entre 2 itérations il est nécessaire de connaître toutes les valeurs (marquage et flot d'accumulations) du voisinage de tous les points.

**A rendre à 18h (Dépôt AccumulationRendu4) : Une archive .tgz contenant à nouveau l'ensemble de vos codes avec un README indiquant le travail réalisé. Vous pouvez également joindre à cette archive un rapport indiquant comment vous réaliseriez cette parallélisation si elle est différente de celle décrite dans le rapport du premier rendu.**
